In [1]:
import glworia.amplification_factor
import glworia.lens_functions
from glworia.amplification_factor import *
from glworia.lens_functions import *
import glworia.lens_functions
from glworia.lens_functions import *
import glworia.contour
from glworia.contour import *
import glworia.utils
from glworia.utils import *
import glworia.root
from glworia.root import *
import glworia.plot
from glworia.plot import *

In [2]:
from importlib import reload

In [3]:
reload(glworia.amplification_factor)
reload(glworia.lens_functions)
from glworia.amplification_factor import *
from glworia.lens_functions import *
reload(glworia.lens_functions)
from glworia.lens_functions import *
reload(glworia.contour)
from glworia.contour import *
reload(glworia.utils)
from glworia.utils import *
reload(glworia.root)
from glworia.root import *
reload(glworia.plot)
from glworia.plot import *

In [4]:
from jax.config import config
config.update("jax_enable_x64", True)

In [5]:
import matplotlib.pyplot as plt
from tqdm import tqdm

In [6]:
T_funcs, helper_funcs = amplification_computation_prep(Psi_NFW)
crit_curve_helper_funcs = make_crit_curve_helper_func(T_funcs)

In [7]:
param_arr = jnp.linspace(0.1, 10., 100000)
crit_funcs = crtical_curve_interpolants(param_arr, T_funcs, crit_curve_helper_funcs)

2023-05-20 19:49:33.098614: W external/xla/xla/service/gpu/nvptx_compiler.cc:564] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.1.105). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [8]:
kappa_low = 0.1
kappa_high = 10.0
y_low = 0.1
y_high = 5.0
N_grid = 15
N_grid_strong = 30
N_crit = 60
T0_max = 1000
N = 200

kappa_arr = jnp.linspace(kappa_low, kappa_high, num = N_grid)
y_arr = jnp.linspace(y_low, y_high, num = N_grid)
kappa_arr_strong = jnp.linspace(kappa_low, kappa_high, num = N_grid_strong)
y_arr_strong = jnp.linspace(y_low, y_high, num = N_grid_strong)

In [9]:
grid_points = make_points_arr_mesh(y_arr, kappa_arr)
grid_points_strong = make_points_arr_mesh(y_arr_strong, kappa_arr_strong)

In [10]:
lens_param_to_y_crit = crit_funcs['lens_param_to_y_crit']
y_crit_to_lens_param = crit_funcs['y_crit_to_lens_param']
y_crit_to_x_crit = crit_funcs['y_crit_to_x_crit']

@partial(jnp.vectorize, signature = '(n,2)->(n)')
def is_strong(point):
    return lens_param_to_y_crit(point[:,1]) > point[:,0]

In [11]:
weak_points = grid_points[~is_strong(grid_points)]
strong_points = grid_points_strong[is_strong(grid_points_strong)]

In [12]:
print(strong_points[50][1])

6.586206896551724


In [13]:
u_strong_min_out_list = []
u_strong_sad_max_list = []

In [14]:
for y0, kappa in tqdm(strong_points):
    y =jnp.array([y0, 0.])
    lens_params = jnp.array([kappa])
    contour_int = amplification_computation_for_interpolation(
                T_funcs, 
                helper_funcs, 
                crit_funcs, 
                y, 
                lens_params, 
                crit_run = False,
                N = N,
                T0_max = T0_max)
    u_strong_min_out_list.append(contour_int.u_min_out)
    u_strong_sad_max_list.append(contour_int.u_sad_max)

100%|██████████| 162/162 [04:41<00:00,  1.74s/it]


In [15]:
strong_points.shape

(162, 2)

In [16]:
kappa_crit_points = jnp.linspace(kappa_low, kappa_high, num = N_crit)
x_crit_points, y_crit_points = get_crit_curve_1D(kappa_crit_points, T_funcs, crit_curve_helper_funcs, x_hi = 100.)

In [17]:
crit_points = jnp.vstack([y_crit_points, kappa_crit_points]).T

In [18]:
boundary_crit_points = jnp.array([[y_arr[0], y_crit_to_lens_param(y_arr[0])],
                                  [y_arr[-1], y_crit_to_lens_param(y_arr[-1])]])
boundary_crit_points = boundary_crit_points[~jnp.isnan(boundary_crit_points).any(axis = 1)]

In [19]:
crit_points_all = jnp.vstack([boundary_crit_points, crit_points])
y_crit_points_all = jnp.append((boundary_crit_points[:,0]), y_crit_points)
x_crit_points_all = jnp.append((y_crit_to_x_crit(boundary_crit_points[:,0])), -x_crit_points)
kappa_crit_points_all = jnp.append((boundary_crit_points[:,1]), kappa_crit_points)

in_bound = crit_points_all[:, 0] >= 0.1

crit_points_in_bound = crit_points_all[in_bound]
y_crit_points_in_bound = y_crit_points_all[in_bound]
x_crit_points_in_bound = x_crit_points_all[in_bound]
kappa_crit_points_in_bound = kappa_crit_points_all[in_bound]

In [20]:
crit_bisect_x_low = -10
crit_bisect_x_high = 10
crit_bisect_x_num = 100
crit_x_init_arr = jnp.linspace(
    crit_bisect_x_low, 
    crit_bisect_x_high, 
    crit_bisect_x_num)

crit_screen_round_decimal = 8

newt_cond_fun = helper_funcs['newt_cond_fun']
newt_step_fun = helper_funcs['newt_step_fun']

get_crit_points_1D_vec = jnp.vectorize(
    get_crit_points_1D, 
    excluded = {0, 1, 2, 5},
    signature = '(),()->(3)')
get_crit_points_2D_arr = lambda x: get_crit_points_1D_vec(
    crit_x_init_arr, 
    newt_cond_fun, 
    newt_step_fun,  
    x[:, 0], x[:, 1], 
    crit_screen_round_decimal)

In [21]:
crit_image_x_newt = get_crit_points_2D_arr(crit_points_in_bound)
crit_sad_x, crit_max_x, crit_min_x = jnp.hsplit(crit_image_x_newt, crit_image_x_newt.shape[1])

In [22]:
crit_image_x = jnp.vstack([x_crit_points_in_bound, x_crit_points_in_bound, crit_min_x.ravel()]).T

In [23]:
T_1D_vec = T_funcs['T_1D_vec']

In [24]:
crit_T_vir = T_1D_vec(x_crit_points_in_bound, crit_points_in_bound[:,0], crit_points_in_bound[:,1], Psi_NFW)

In [25]:
crit_sad_x, crit_max_x, crit_min_x = jnp.hsplit(crit_image_x, crit_image_x.shape[1])

In [26]:
u_crit_min_out_list = []
u_crit_sad_max_list = []
mu_min_crit_list = []

In [27]:
for y0, kappa in tqdm(crit_points_in_bound):
    y =jnp.array([y0, 0.])
    lens_params = jnp.array([kappa])
    contour_int = amplification_computation_for_interpolation(
                T_funcs, 
                helper_funcs, 
                crit_funcs, 
                y, 
                lens_params, 
                crit_run = True,
                N = N,
                T0_max = T0_max)
    u_crit_min_out_list.append(contour_int.u_min_out)
    u_crit_sad_max_list.append(contour_int.u_sad_max)
    mu_min_crit_list.append(contour_int.mu_min)

In [28]:
u_strong_full_min_out = jnp.array(u_strong_min_out_list + u_crit_min_out_list)
u_strong_full_sad_max = jnp.array(u_strong_sad_max_list + u_crit_sad_max_list)
strong_full_points = jnp.vstack([strong_points, crit_points_in_bound])

In [29]:
from scipy.interpolate import LinearNDInterpolator

In [30]:
u_points_low = u_strong_full_min_out[:,0,:]
u_points_mid_1 = u_strong_full_min_out[:,1,:]
u_points_mid_2 = u_strong_full_min_out[:,2,:]
u_points_high = u_strong_full_min_out[:,3,:]

In [31]:
interp_low = LinearNDInterpolator(strong_full_points, u_points_low)
interp_mid_1 = LinearNDInterpolator(strong_full_points, u_points_mid_1)
interp_mid_2 = LinearNDInterpolator(strong_full_points, u_points_mid_2)
interp_high = LinearNDInterpolator(strong_full_points, u_points_high)
interp_sad_max = LinearNDInterpolator(strong_full_points, u_strong_full_sad_max)

In [32]:
import pickle
import os

interpolate_dir_name = f'y_{y_low:.3f}_{y_high:.3f}_kappa_{kappa_low:.3f}_{kappa_high:.3f}_N_grid_{N_grid}_N_grid_strong_{N_grid_strong}_N_crit_{N_crit}_N_{N}'

os.makedirs(f'./interpolation/{interpolate_dir_name}', exist_ok = True)

with open(f'./interpolation/{interpolate_dir_name}/interp_strong_low.pkl', 'wb') as f:
    pickle.dump(interp_low, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_strong_mid_1.pkl', 'wb') as f:
    pickle.dump(interp_mid_1, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_strong_mid_2.pkl', 'wb') as f:
    pickle.dump(interp_mid_2, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_strong_high.pkl', 'wb') as f:
    pickle.dump(interp_high, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_strong_sad_max.pkl', 'wb') as f:
    pickle.dump(interp_sad_max, f)

In [41]:
u_weak_min_out_list = []
T_vir_list = []
mu_min_weak_list = []

In [42]:
for y0, kappa in tqdm(weak_points):
    y =jnp.array([y0, 0.])
    lens_params = jnp.array([kappa])
    contour_int = amplification_computation_for_interpolation(
                T_funcs, 
                helper_funcs, 
                crit_funcs, 
                y, 
                lens_params, 
                crit_run = False,
                N = N,
                T0_max = T0_max)
    u_weak_min_out_list.append(contour_int.u_min_out)
    T_vir_list.append(contour_int.T_vir)
    mu_min_weak_list.append(contour_int.mu_min)

100%|██████████| 182/182 [03:57<00:00,  1.31s/it]


In [43]:
u_weak_full_min_out = jnp.array(u_weak_min_out_list + u_crit_min_out_list)
weak_full_points = jnp.vstack([weak_points, crit_points_in_bound])
T_vir_full = jnp.array(T_vir_list + crit_T_vir.tolist())
mu_min_weak_full = jnp.array(mu_min_weak_list + mu_min_crit_list)

In [44]:
u_weak_points_low = u_weak_full_min_out[:,0,:]
u_weak_points_mid_1 = u_weak_full_min_out[:,1,:]
u_weak_points_mid_2 = u_weak_full_min_out[:,2,:]
u_weak_points_high = u_weak_full_min_out[:,3,:]

In [45]:
interp_weak_low = LinearNDInterpolator(weak_full_points, u_weak_points_low)
interp_weak_mid_1 = LinearNDInterpolator(weak_full_points, u_weak_points_mid_1)
interp_weak_mid_2 = LinearNDInterpolator(weak_full_points, u_weak_points_mid_2)
interp_weak_high = LinearNDInterpolator(weak_full_points, u_weak_points_high)
interp_T_vir = LinearNDInterpolator(weak_full_points, T_vir_full)
interp_mu_min_weak = LinearNDInterpolator(weak_full_points, mu_min_weak_full)

In [46]:
with open(f'./interpolation/{interpolate_dir_name}/interp_weak_low.pkl', 'wb') as f:
    pickle.dump(interp_weak_low, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_weak_mid_1.pkl', 'wb') as f:
    pickle.dump(interp_weak_mid_1, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_weak_mid_2.pkl', 'wb') as f:
    pickle.dump(interp_weak_mid_2, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_weak_high.pkl', 'wb') as f:
    pickle.dump(interp_weak_high, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_T_vir.pkl', 'wb') as f:
    pickle.dump(interp_T_vir, f)

with open(f'./interpolation/{interpolate_dir_name}/interp_mu_min_weak.pkl', 'wb') as f:
    pickle.dump(interp_mu_min_weak, f)

In [54]:
reload(glworia.amplification_factor)
reload(glworia.lens_functions)
from glworia.amplification_factor import *
from glworia.lens_functions import *
reload(glworia.lens_functions)
from glworia.lens_functions import *
reload(glworia.contour)
from glworia.contour import *
reload(glworia.utils)
from glworia.utils import *
reload(glworia.root)
from glworia.root import *
reload(glworia.plot)
from glworia.plot import *